# Photometry Homework

Obs Tech - Homework 2
21 October 2020
Due: Thursday (November 5)


The primary goal of this homework is for the student to become familiar with the tools for photometry that will be needed for their class project. 

In [1]:
from astropy.io import fits
from glob import glob
import numpy as np
import matplotlib.pyplot as plt
from scipy.optimize import curve_fit

## Step 1: Image processing

In [2]:
def darks(path):
    """
    Didn't end up using this function
    
    Parameters
    ----------
    path: str
        path of all dark frames from rho
    Returns
    -------
    darks: np.ndarray
        3d array of dark frames
    """
    
    darks_filenames = glob(path)
    darks = []
    darks_filters = []

    for i in range(len(darks_filenames[1:])):
        darks.append(fits.getdata(darks_filenames[i]))
        hdul = fits.open(darks_filenames[i])
        darks_filters.append(hdul[0].header['FILTER'])
        hdul.close()

    darks = np.asarray(darks)
    darks = darks.astype(float)

In [3]:
def get_flats(path):
    """
    Extracts flats from fits files
    
    Parameters
    ----------
    path: str
        path of all flat frames from rho
    Returns
    -------
    X_flats: np.ndarray
        3d array of flat frames in each filter (R, V, B, I)
    """

    flats_filenames = glob(path)
    flats = []
    flats_filters = []

    for i in range(len(flats_filenames[1:])):
        flats.append(fits.getdata(flats_filenames[i]))
        hdul = fits.open(flats_filenames[i])
        flats_filters.append(hdul[0].header['FILTER'])
        hdul.close()

    flats_filters = np.asarray(flats_filters)
    flats = np.asarray(flats)
    flats = flats.astype(float)

    R_flats = flats[np.where(flats_filters == 'R')]
    V_flats = flats[np.where(flats_filters == 'V')]
    B_flats = flats[np.where(flats_filters == 'B')]
    I_flats = flats[np.where(flats_filters == 'I')]
    
    return R_flats, V_flats, B_flats, I_flats

In [4]:
_, V_flats, _, _ = get_flats('2020-10-17/h_persei_flats*')

In [5]:
def seq(path, expt):
    """
    Extracts light frames & darks from fits files
    
    Parameters
    ----------
    path: str
        Path of all flat frames from rho
    expt: float
        Exposure time of darks
    Returns
    -------
    X_seq: np.ndarray
        3d array of light frames in each filter (R, V, B, I)
    seq_darks: np.ndarra
        3d array of dark frames
    """


    seq_filenames = np.sort(glob(path))
    seq_light = []
    seq_darks = []
    seq_filters = []

    for i in range(len(seq_filenames[1:])):

        hdul = fits.open(seq_filenames[i])
        if hdul[0].header['FRAME'] == 'Light':
            seq_filters.append(hdul[0].header['FILTER'])
            seq_light.append(fits.getdata(seq_filenames[i]))
        else:
            seq_darks.append(fits.getdata(seq_filenames[i]))
        hdul.close()

    seq_filters = np.asarray(seq_filters)
    seq_light = np.asarray(seq_light)
    seq_light = seq_light.astype(float)


    R_seq = seq_light[np.where(seq_filters == 'R')]
    V_seq = seq_light[np.where(seq_filters == 'V')]
    B_seq = seq_light[np.where(seq_filters == 'B')]
    I_seq = seq_light[np.where(seq_filters == 'I')]
    
    return R_seq, V_seq, B_seq, I_seq, seq_darks

In [6]:
R_seq, V_seq, B_seq, I_seq, seq_dark = seq('2020-10-17/h_persei_seq*', 5.0)

### Step 2: Master Dark:

In [7]:
master_V_dark = np.median(seq_dark, axis=0)

### Step 3: Master Flat:

In [8]:
comb_V_flat = np.median(V_flats,axis=0) - master_V_dark
div = master_V_dark/np.median(comb_V_flat)
master_V_flat = comb_V_flat - div

### Step 4, 5: Subtract dark and divide by flat for each science frame

In [9]:
V_proc = []

for i in range(len(V_seq)):
    V_proc_temp = V_seq[i] - master_V_dark
    V_proc.append(V_proc_temp/master_V_flat)

### Step 6: Shifting frames with np.roll

In [24]:
shifts = [[0,4], [0,7], [0,7], [-1,8], [-2,7], [-3,5], [-6,3], [-8,2], [-10,-1]]



In [40]:
def shiftroll(shifts, frames):
    """
    Shifts off-center frames, median-combines, and flips along y-axis
    
    Parameters
    ----------
    shifts: list
        list of [x,y] shifts
    frames: np.ndarray
        3d array of processed science images
    Returns
    -------
    rolledframes: np.ndarray
        Final median-combined image
    """
    rolledframes = []
    rolledframes.append(V_seq[0])
    for i in range(1, len(frames)):
        rolledframes.append(np.roll(frames[i], shifts[i-1], axis=(0,1)))
    
    rolledframes = np.asarray(rolledframes)
    rolledframes = np.median(rolledframes, axis=0)
    
    #flip to match DS9 orientation
    rolledframes = np.flip(rolledframes, 0)
    
    return rolledframes

In [41]:
V = shiftroll(shifts, V_proc)

## Aperture Photometry

In [43]:
from astropy.modeling import models, fitting

In [44]:
def cutout(image, bb, center):
    """
    Cuts out smaller frame around star
    
    Parameters
    ----------
    image: np.ndarray
        Whole image
    bb: int
        Cut out a box of side size 2*bb
    center: [x,y]
        Center of star in whole image [x,y]
    
    Returns
    -------
    xp, yp: ints
        Grids of box size
    box: np.ndarray
        Cutout image
    """
    xc = int(center[0])
    yc = int(center[1])
    box = image[yc-bb:yc+bb, xc-bb:xc+bb]
    xp, yp = box.shape
    return xp, yp, box

In [45]:
def gaussian(yp, xp, box):
    """ Fits a 2d Gaussian to star
    """
    y, x, = np.mgrid[:yp, :xp]
    f_init = models.Gaussian2D()
    fit_f = fitting.LevMarLSQFitter()
    fV = fit_f(f_init, x, y, box)
    
    return fV, x, y

In [46]:
def sigma_avg(x,y):
    """ Averages sigmas
    """
    return np.mean([float(x), float(y)])

In [47]:
def px_to_arcsec(fwhm):
    return fwhm*0.605

Star 1: HD 14052

(2.307778, 57.20833)
http://simbad.u-strasbg.fr/simbad/sim-basic?Ident=tyc+3694-3626-1&submit=SIMBAD+search
    
Star 2: BD+56 511

(2.313056, 57.06722)
http://simbad.u-strasbg.fr/simbad/sim-basic?Ident=tyc+3694-2565-1&submit=SIMBAD+search

In [48]:
def psf(image, targ_loc, boxsize, expt, aper, aper_inside, aper_outside, plot='star'):
    """
    Star psf
    
    Parameters
    ----------
    image: np.ndarray
        Entire image
    targ_loc: [x,y]
        Coordinates of target
    boxsize: int
        1/2 box side length for image cutout around target star
    expt: float
        Exposure time
    aper: float
        Aperture size (units of FWHM)
    aper_inside: float
        Inner aperture size (units of FWHM)
    aper_outside: float
        Outside aperture size (units of FWHM)
    plot: 'star', 'gaussian', 'cutout'
        Plot keyword
    
    Returns
    -------
    F_targ: float
        Flux in PSF of target
    F_targ_persec: float
        Flux in PSF of target per second of exposure
    """
    F_sky_pix = []
    
    targ_loc[1] = 1266-targ_loc[1]
    
    xp, yp, current_image = cutout(image, boxsize, targ_loc)
    
    fV, x, y = gaussian(xp, yp, current_image)
    
    #plt.xlabel('x pixels')
    #plt.ylabel('y pixels')
    
    if plot=='star':
        plt.imshow(current_image)
        plt.show()
    elif plot =='gaussian':
        plt.imshow(fV(x,y))
        plt.show()
    
    sigma = sigma_avg(fV.x_stddev[0],fV.y_stddev[0])
    #xbar = fV.x_mean[0]
    fwhm = 2.355*float(sigma)
    fit = fV(x, y)
    
    center = np.where(fit == np.max(fit))
 
    circ_image = np.zeros(current_image.shape)
    N_targ = 0
    for i in range(len(current_image)):
        for j in range(len(current_image[0])):
            dist = np.sqrt((center[0]-i)**2+(center[1]-j)**2)
            if dist >= aper_inside*fwhm and dist <= aper_outside*fwhm:
                F_sky_pix.append(current_image[i][j])  
            if (dist > aper*fwhm):
                circ_image[i][j] = 0.0
            else:
                circ_image[i][j] = current_image[i][j]
                N_targ+=1

    F_sky_pix_med = np.median(F_sky_pix)
    F_targ_raw = 0
    F_targ_raw = np.sum(circ_image)
      
    if plot=='cutout':
        plt.imshow(circ_image)
        plt.show()
    
    F_targ = F_targ_raw - N_targ*F_sky_pix_med
    F_targ_persec = F_targ/expt
                        
    return F_targ_persec

In [49]:
tnames = ['2-3694-3626-1', '2-3694-2565-1', '2-3694-2943-1', '2-3694-1712-1', '2-3694-3750-1', '2-3694-3451-1', '2-3694-1822-1', '2-3694-1719-1',  '2-3694-1853-1', '2-3694-1363-1']
targlocs = [[440, 134], [382,1013], [529, 634], [892, 1139], [194, 272], [329,956], [1150, 146], [1032, 812], [1463, 134], [1485, 1077]]
boxsizes = [25, 35, 45, 30, 36, 40, 40, 27, 20, 56] 

In [53]:
def get_flux_per_s(image, targlocs, boxsizes, aper, aper_in, aper_out, plot=None):
    
    tnames = ['2-3694-3626-1', '2-3694-2565-1', '2-3694-2943-1', '2-3694-1712-1', '2-3694-3750-1', '2-3694-3451-1', '2-3694-1822-1', '2-3694-1719-1',  '2-3694-1853-1', '2-3694-1363-1']
    targlocs = [[440, 134], [382,1013], [529, 634], [892, 1139], [194, 272], [329,956], [1150, 146], [1032, 812], [1463, 134], [1485, 1077]]
    boxsizes = [25, 35, 45, 30, 36, 40, 40, 27, 20, 56]
    
    Flux_per_s = [0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
    
    for i in range(len(targlocs)):
        F_targ_pers = psf(image, targlocs[i], boxsizes[i], 5.0, aper, aper_in, aper_out, plot)
        Flux_per_s[i] = F_targ_pers
    return Flux_per_s

In [61]:
Flux_per_s = get_flux_per_s(V, targlocs, boxsizes, 1.3, 2., 3., plot=None)


### Zeropoint magnitude

In [62]:
cat = [7.7866, 8.2215, 8.3765, 10.17, 10.219, 10.6020, 11.1, 8.4634, 9.3223, 11.320]

In [63]:
def m_i(flux):
    return -2.5*np.log10(flux)

def dm_std(m_cat_std, flux_std):
    dm = m_cat_std - m_i(flux_std)
    return dm

def m_obs(dm, flux_targ):
    m_obs = m_i(flux_targ) + dm
    return m_obs

def mag_diffs(Flux_per_s, cat_mags):
    dm = dm_std(cat, Flux_per_s)
    dm = np.mean(dm)
    obs_mag = m_obs(dm, Flux_per_s)
    mag_diff = cat - obs_mag
    return mag_diff, obs_mag

# Simbad - calibrated magnitude for four configurations
The three inputs are: inner apperature, sky 'donut' inner radius and outter radius. \
The calibration process happens for every star (dm is calculated for each star) and then the average dm is taken. This is then used to calibrate observed fluxes and compare them
to simbad magnitudes.

In [64]:
tnames = ['2-3694-3626-1', '2-3694-2565-1', '2-3694-2943-1', '2-3694-1712-1', '2-3694-3750-1', '2-3694-3451-1', '2-3694-1822-1', '2-3694-1719-1',  '2-3694-1853-1', '2-3694-1363-1']
targlocs = [[440, 134], [382,1013], [529, 634], [892, 1139], [194, 272], [329,956], [1150, 146], [1032, 812], [1463, 134], [1485, 1077]]
boxsizes = [25, 35, 45, 30, 36, 40, 40, 27, 20, 56]

In [65]:
for i in range(10):
    psf(V, targlocs[i], boxsizes[i], 5.0, 1.3, 2., 3., plot=None)
    

In [66]:
Flux_per_s = get_flux_per_s(V, targlocs, boxsizes, 1.3, 2., 3., plot=None)
mag_diffs(Flux_per_s, cat)

(array([-0.1420963 , -0.69694104, -0.78505123,  0.85820118,  0.39024291,
         0.30116784,  0.79194049, -0.690147  , -0.30504157,  0.27772473]),
 array([ 7.9286963 ,  8.91844104,  9.16155123,  9.31179882,  9.82875709,
        10.30083216, 10.30805951,  9.153547  ,  9.62734157, 11.04227527]))

In [67]:
mag_diffs(get_flux_per_s(V, targlocs, boxsizes, 0.5, 2., 3., plot=None), cat)

(array([-0.18966458, -0.66495173, -0.78578795,  0.89228773,  0.42877572,
         0.33109339,  0.80423963, -0.70869269, -0.36204384,  0.25474433]),
 array([ 7.97626458,  8.88645173,  9.16228795,  9.27771227,  9.79022428,
        10.27090661, 10.29576037,  9.17209269,  9.68434384, 11.06525567]))

In [68]:
mag_diffs(get_flux_per_s(V, targlocs, boxsizes, 2., 2., 3., plot=None), cat)

(array([-0.14069752, -0.70048745, -0.77804794,  0.85286101,  0.38676549,
         0.29955205,  0.80207043, -0.68584414, -0.31702419,  0.28085225]),
 array([ 7.92729752,  8.92198745,  9.15454794,  9.31713899,  9.83223451,
        10.30244795, 10.29792957,  9.14924414,  9.63932419, 11.03914775]))

In [69]:
mag_diffs(get_flux_per_s(V, targlocs, boxsizes, 1.3, 1.5, 3., plot=None), cat)


(array([-0.1401819 , -0.69634767, -0.78414744,  0.85831279,  0.39009491,
         0.30023319,  0.79088697, -0.68895719, -0.30612013,  0.27622647]),
 array([ 7.9267819 ,  8.91784767,  9.16064744,  9.31168721,  9.82890509,
        10.30176681, 10.30911303,  9.15235719,  9.62842013, 11.04377353]))